In [6]:
from datasets import load_dataset
from transformers import pipeline
from sklearn.metrics import accuracy_score, log_loss, classification_report
import numpy as np
import pandas as pd

In [7]:
url = "https://raw.githubusercontent.com/mhjabreel/CharCnn_Keras/master/data/ag_news_csv/test.csv"
df = pd.read_csv(url, header=None, names=["label","title","description"])

In [8]:
df["text"] = df["title"] + ". " + df["description"]
df = df.sample(n=500, random_state=42).reset_index(drop=True)

texts = df["text"].tolist()
# в CSV метки от 1 до 4, переводим в 0–3
y_true = df["label"].values - 1

# 2. Определяем имена классов
candidate_labels = ["World", "Sports", "Business", "Sci/Tech"]

In [10]:
classifier = pipeline(
        "zero-shot-classification",
        model="valhalla/distilbart-mnli-12-1",  # компактная модель на NLI
        device=-1  # CPU; для GPU укажите device=0
    )
candidate_labels = ["World", "Sports", "Business", "Sci/Tech"]

Device set to use cpu


In [11]:
soft_preds = []   # накапливаем прогнозы вероятностей
hard_preds = []   # накапливаем класс argmax
for text in texts:
    res = classifier(text, candidate_labels)
    # преобразуем в вектор вероятностей в порядке candidate_labels
    score_map = dict(zip(res["labels"], res["scores"]))
    probs = [score_map[lbl] for lbl in candidate_labels]
    soft_preds.append(probs)
    hard_preds.append(res["labels"][0])

soft_preds = np.array(soft_preds)
hard_pred_ids = np.array([candidate_labels.index(lbl) for lbl in hard_preds])


In [12]:
ll = log_loss(y_true, soft_preds, labels=list(range(len(candidate_labels))))
acc = accuracy_score(y_true, hard_pred_ids)

print(f"Log Loss (soft predictions): {ll:.4f}")
print(f"Accuracy  (hard predictions): {acc:.4f}\n")

print("Classification report (hard predictions):")
print(classification_report(
    y_true,
    hard_pred_ids,
    target_names=candidate_labels,
    digits=4
))

Log Loss (soft predictions): 0.8414
Accuracy  (hard predictions): 0.7240

Classification report (hard predictions):
              precision    recall  f1-score   support

       World     0.6154    0.7934    0.6931       121
      Sports     0.9333    0.9655    0.9492       116
    Business     0.6188    0.7984    0.6972       124
    Sci/Tech     0.8594    0.3957    0.5419       139

    accuracy                         0.7240       500
   macro avg     0.7567    0.7382    0.7203       500
weighted avg     0.7578    0.7240    0.7115       500



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:3001: UserWarning: The y_pred values do not sum to one. Make sure to pass probabilities.
  warnings.warn(


Log Loss (soft predictions): 0.8414

  -	Значение ниже 1 говорит о том, что модель в целом выставляет умеренные вероятности: она не слишком «уверена» в каждом своём ответе, но и не даёт слишком размытые распределения.

  -	Есть пространство для улучшения калибровки (например, с помощью temperature scaling или небольшого дообучения).

Accuracy (hard predictions): 0.7240

  - Модель верно угадала тему примерно в 72,4% случаев без какой-либо донастройки на конкретный датасет. Для zero-shot это вполне хороший результат.


Итоги:

Zero-shot без дообучения даёт приемлемый baseline (~72% точности), но остаётся значительный разрыв до supervised-классификаторов, которые на AG News обычно достигают >90%
